# Satellite Imagery from Sentinel 2

Before running the notebook, do the following steps first (following instructions and code [here](https://github.com/artefactory/medium_satellite_imagery)):

*   create a geojson file of the zone you want a satellite imagery of and save it in 1_collection/city.geojson
*   create an account at Coppernicus Open Access Hub. Write the username and the password in a json file and save it in 1_collection/sentinel_api_credentials.json. Format: {"username": "XXXX", "password": "XXXX"}




## Step 0: Requirements & Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install rasterio
!pip install sentinelsat

In [ ]:
import os 
import cv2
from datetime import date
import json
import itertools
import rasterio
import matplotlib.pyplot as plt
import numpy as np
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
import pandas as pd

## Step 1: Authentication with Copernicus Open Access Hub

In [ ]:
with open("./sentinel_api_credentials.json", "r") as infile:
    credentials = json.load(infile)

api = SentinelAPI(credentials["username"], credentials["password"], "https://scihub.copernicus.eu/dhus")

## Step 2: Define area of interest in a geojson file

A way to specify the zones you want to explore is to provide the API with a geojson file, which is a json containing GPS coordinates position (latitude and longitude) of a zone. Geojson can be quickly created on this [website](https://geojson.io/#map=2/20/0).

In [ ]:
geojson_path = "./lviv.geojson"

## Step 3: Query satellite images via Sentinelsat API

In [ ]:
shape = geojson_to_wkt(read_geojson(geojson_path))

images = api.query(
    shape,
    date=(date(2021, 9, 1), date(2022, 1, 30)),
    platformname="Sentinel-2",
    processinglevel = "Level-2A",
    cloudcoverpercentage=(0, 5)
)

images_df = api.to_dataframe(images)

We can use the is_online() function from the sentinelsat package to filter the queried images that are available online. A for loop iterates over the images dictionary and checks if each image is online. 

In [ ]:
online_images = {}
for id, item in images.items():
    if api.is_online(id):
        online_images[id] = item

# Convert the filtered images to a dataframe
online_images_df = api.to_dataframe(online_images)

In [ ]:
key_columns = ["platformserialidentifier", "processinglevel", "cloudcoverpercentage", 
               "vegetationpercentage", "beginposition"]
online_images_df = online_images_df.sort_values(by=['cloudcoverpercentage'], ascending=True)
online_images_df[key_columns].head(5)

##  Step 4: Download satellite image

In [ ]:
uuid = "36e3b1a9-0023-4e2b-adde-ba7f1e5337a3"
api.download(uuid)

MD5 checksumming:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

{'id': '36e3b1a9-0023-4e2b-adde-ba7f1e5337a3',
 'title': 'S2A_MSIL2A_20211008T093041_N0301_R136_T34UGA_20211008T121442',
 'size': 1161828733,
 'md5': '97045258b1238b106c0dcf45eed9147b',
 'date': datetime.datetime(2021, 10, 8, 9, 30, 41, 24000),
 'footprint': 'POLYGON((23.82076894501528 50.51811370350305,25.365742763507004 50.47036063098581,25.277370381662823 49.48562029230301,23.763545147601363 49.53174440621896,23.82076894501528 50.51811370350305))',
 'url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('36e3b1a9-0023-4e2b-adde-ba7f1e5337a3')/$value",
 'Online': True,
 'Creation Date': datetime.datetime(2021, 10, 8, 18, 16, 58, 220000),
 'Ingestion Date': datetime.datetime(2021, 10, 8, 17, 13, 2, 468000),
 'manifest_name': 'manifest.safe',
 'product_root_dir': 'S2A_MSIL2A_20211008T093041_N0301_R136_T34UGA_20211008T121442.SAFE',
 'quicklook_url': "https://scihub.copernicus.eu/dhus/odata/v1/Products('36e3b1a9-0023-4e2b-adde-ba7f1e5337a3')/Products('Quicklook')/$value",
 'path': '

## Step 5: Unzip satellite image and save to folder

In [ ]:
!unzip /content/S2A_MSIL2A_20211008T093041_N0301_R136_T34UGA_20211008T121442.zip -d ../data/sentinel2/lviv

# Credits

*   Antoine Aubay (2021): Leveraging satellite imagery for machine learning computer vision applications. Available at: https://medium.com/artefact-engineering-and-data-science/leveraging-satellite-imagery-for-machine-learning-computer-vision-applications-d22143f72d94

